## Data


In [1]:
IGNORE_INDEX = -100
MAX_LENGTH = 128

In [2]:
from datasets import load_dataset

wnut_dataset = load_dataset("wnut_17", trust_remote_code=True)

/home/solan/repos/entity-extraction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wnut_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [4]:
tokens = wnut_dataset["train"][0]["tokens"]
ner_tags = wnut_dataset["train"][0]["ner_tags"]
assert len(tokens) == len(ner_tags)
tokens[:5], ner_tags[:5]

(['@paulwalk', 'It', "'s", 'the', 'view'], [0, 0, 0, 0, 0])

In [5]:
label_list = wnut_dataset["train"].features["ner_tags"].feature.names
label_list, len(label_list)

(['O',
  'B-corporation',
  'I-corporation',
  'B-creative-work',
  'I-creative-work',
  'B-group',
  'I-group',
  'B-location',
  'I-location',
  'B-person',
  'I-person',
  'B-product',
  'I-product'],
 13)

In [6]:
from transformers import AutoTokenizer

model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
example = wnut_dataset["train"][0]
tokenized_input = tokenizer(
    example["tokens"], max_length=MAX_LENGTH, truncation=True, is_split_into_words=True
)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:5], example["tokens"][:5], example["ner_tags"][:5]

(['[CLS]', '@', 'paul', '##walk', 'it'],
 ['@paulwalk', 'It', "'s", 'the', 'view'],
 [0, 0, 0, 0, 0])

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        max_length=MAX_LENGTH,  #'max_length',
        is_split_into_words=True,
    )

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        # Map tokens to their respective word.
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx and word_ids != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(IGNORE_INDEX)

            # if word_idx is None:
            #     label_ids.append(IGNORE_INDEX)
            # elif (
            #     word_idx != previous_word_idx
            # ):  # Only label the first token of a given word.
            #     label_ids.append(label[word_idx])
            # else:
            #     label_ids.append(IGNORE_INDEX)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [9]:
tokenized_wnut = wnut_dataset.map(tokenize_and_align_labels, batched=True)

In [10]:
row = tokenized_wnut["train"][0]

print(len(row["tokens"]), len(row["ner_tags"]))
print(len(row["input_ids"]), len(row["attention_mask"]), len(row["labels"]))
row.keys()

27 27
34 34 34


dict_keys(['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'])

In [11]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, padding="max_length", max_length=MAX_LENGTH
)

## Metrics


In [12]:
import evaluate

# https://huggingface.co/spaces/evaluate-metric/seqeval
seqeval = evaluate.load("seqeval")

In [13]:
# labels = [label_list[i] for i in example[f"ner_tags"]]
# labels

In [14]:
import numpy as np

# labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)  # 2
    # print(predictions)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != IGNORE_INDEX]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != IGNORE_INDEX]
        for prediction, label in zip(predictions, labels)
    ]
    # print(true_predictions)
    # print(true_labels)

    results = seqeval.compute(
        predictions=true_predictions, references=true_labels, zero_division=0
    )
    # print(results)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [15]:
# p = ([[[0, 0, 1], [0, 1, 0]]], [2, 1])
# compute_metrics(p)

## Model


In [16]:
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}
id2label = {v: k for k, v in label2id.items()}

In [17]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=13,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from torchinfo import summary

summary(model, verbose=1)
None

Layer (type:depth-idx)                                  Param #
DistilBertForTokenClassification                        --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           9,997
Total params: 66,372,877
Trainable params: 66,372,877
Non-trainable params: 0


## LoRA


In [19]:
from peft import LoraConfig, get_peft_model

target_modules = ["q_lin", "k_lin", "v_lin"]

# Load config
# Write your code here
config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
    bias="none",
    target_modules=target_modules,
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
summary(lora_model, verbose=1)
None

Layer (type:depth-idx)                                                 Param #
PeftModel                                                              --
├─LoraModel: 1-1                                                       --
│    └─DistilBertForTokenClassification: 2-1                           --
│    │    └─DistilBertModel: 3-1                                       68,132,352
│    │    └─Dropout: 3-2                                               --
│    │    └─ModulesToSaveWrapper: 3-3                                  19,994
Total params: 68,152,346
Trainable params: 1,779,469
Non-trainable params: 66,372,877


## ClearML


In [20]:
import os
from dotenv import load_dotenv
from IPython.display import clear_output

load_dotenv(".env")

CLEARML_API_ACCESS_KEY = os.getenv("CLEARML_API_ACCESS_KEY")
CLEARML_API_SECRET_KEY = os.getenv("CLEARML_API_SECRET_KEY")

%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml/
%env CLEARML_FILES_HOST=https://files.clear.ml/
%env CLEARML_API_ACCESS_KEY=$CLEARML_API_ACCESS_KEY
%env CLEARML_API_SECRET_KEY=$CLEARML_API_SECRET_KEY
clear_output()

In [21]:
from clearml import Task

# default is ViT if input is empty
if not (task_name := input()):
    task_name = "BERT"

task = Task.init(
    task_name=task_name,
    project_name="PMLDL",
)

ClearML Task: created new task id=ba411791de9f4632a43f00fabd5ff593
2024-10-28 16:02:12,089 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-10-28 16:02:12,107 - clearml.Repository Detection - WARNING - Please install nbconvert using "pip install nbconvert"
2024-10-28 16:02:12,126 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/3fdff4c44fe4469db0bf496ddae269dc/experiments/ba411791de9f4632a43f00fabd5ff593/output/log
2024-10-28 16:02:13,014 - clearml.log - WARNING - Event reporting sub-process lost, switching to thread based reporting


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Training


In [24]:
batch_size = 32
epochs = 20

training_args = TrainingArguments(
    # output and logging
    output_dir="bert-output",
    # overwrite_output_dir=True,
    logging_steps=10,
    # remove_unused_columns=False,
    push_to_hub=False,
    report_to="clearml",
    # data
    label_names=["labels"],  # https://github.com/huggingface/transformers/issues/28530
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # hyperparameters
    learning_rate=2e-5,
    num_train_epochs=epochs,
    gradient_accumulation_steps=2,
    weight_decay=0.001,
    # strategies
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # other
    seed=42,
)

In [25]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],
)

In [26]:
trainer.train()

2024-10-28 16:02:34,387 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


  1%|          | 10/1060 [00:04<07:40,  2.28it/s]

{'loss': 4.8553, 'grad_norm': 13.343891143798828, 'learning_rate': 1.9811320754716984e-05, 'epoch': 0.19}


  2%|▏         | 20/1060 [00:09<07:25,  2.33it/s]

{'loss': 4.6007, 'grad_norm': 13.306522369384766, 'learning_rate': 1.9622641509433963e-05, 'epoch': 0.37}


...

                                                  
 61%|██████    | 642/1060 [08:38<02:34,  2.71it/s]

{'eval_loss': 0.38028204441070557, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.9206161479216546, 'eval_runtime': 4.4491, 'eval_samples_per_second': 289.271, 'eval_steps_per_second': 9.215, 'epoch': 12.0}


 61%|██████    | 642/1060 [08:55<05:48,  1.20it/s]

{'train_runtime': 541.3428, 'train_samples_per_second': 125.392, 'train_steps_per_second': 1.958, 'train_loss': 0.8518582899993825, 'epoch': 12.0}


TrainOutput(global_step=642, training_loss=0.8518582899993825, metrics={'train_runtime': 541.3428, 'train_samples_per_second': 125.392, 'train_steps_per_second': 1.958, 'total_flos': 1386234650382336.0, 'train_loss': 0.8518582899993825, 'epoch': 12.0})